In [4]:
cd /Users/williamharrigan/Desktop/UH/Year_2/Research/mutual_information/ipynb/ 

/Users/williamharrigan/Desktop/UH/Year_2/Research/mutual_information/ipynb


In [5]:
from functools import partial
from Bio import	SeqIO
from soft_align import *
import numpy as np
from collections import Counter
import glob
import pickle
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [6]:
data_dir = '/Users/williamharrigan/Desktop/UH/Year_2/Research/mutual_information/POL/'
embeddings_dir = '/Users/williamharrigan/Desktop/UH/Year_2/Research/mutual_information/POL/polembeddings/'

In [7]:
seqs = SeqIO.to_dict(SeqIO.parse(f"{data_dir}93_pol_A_seqs.fasta", "fasta"))
seq_names = list(seqs.keys())

In [ ]:
for i, seq_id_1 in enumerate(seq_names):
    seq_1_embeddings = torch.load(f"{embeddings_dir}/{seq_id_1}.pt")
    seq_1_str = str(seqs[seq_id_1].seq)
    vals = []
    for seq_id_2 in seq_names[i+1:]:
        for pos in all_matches[seq_id_1]:
            seq_2_embeddings = torch.load(f"{embeddings_dir}/{seq_id_2}.pt")
            seq_2_str =  str(seqs[seq_id_2].seq)
            for search_position in list(all_matches[seq_id_1].keys()):
                match_pos = find_homologous_pos(seq_1_str, pos, seq_2_str, seq_1_embeddings, seq_2_embeddings)
            if match_pos is not None:
                all_matches[seq_id_1][pos][seq_id_2] = match_pos
                all_matches[seq_id_2][match_pos][seq_id_1] = pos

In [366]:
def get_seq_w_max_diff(data_wo_lowhits):
    all_matches_w_count = {}
    ranked = {}
    for k, v in data_wo_lowhits.items():
        all_matches_w_count[k] = {x : len(y) for x, y in v.items()}
    
    for k, v in all_matches_w_count.items():
        if len(v) > 1:
            max_value = max(v.values())
            min_value = min(v.values())
            ranked[k] = {'rank': max_value-min_value, 'pos_w_min': min(v, key=v.get)}

    ranked = {k: v for k, v in sorted(ranked.items(), key=lambda item: item[1]['rank'], reverse=True)}
    seq_to_rm = {}
    for k, v in list(ranked.items())[:1]:
        seq_to_rm = { k: v }
    
    return seq_to_rm


In [389]:
def has_multiple_sites(data_wo_lowhits):
    multiple_sites = False;
    for sequence in data_wo_lowhits:
        if len(data_wo_lowhits[sequence].keys()) > 1:
            multiple_sites = True
            break
    
    return multiple_sites
            

In [491]:
data_wo_lowhits = all_matches

while has_multiple_sites(data_wo_lowhits):
    count+=1
    seq_to_rm = get_seq_w_max_diff(data_wo_lowhits)
    seq_id = list(seq_to_rm.keys())[0]
    pos_w_min = seq_to_rm[seq_id]['pos_w_min']
    print(seq_to_rm)
    print()
    if pos_w_min in data_wo_lowhits[seq_id].keys(): 
        del data_wo_lowhits[seq_id][pos_w_min]
    for k,v in data_wo_lowhits.items():
        for site,hits in v.items():
            if seq_id in hits.keys():
                if hits[seq_id] == pos_w_min:
                    del data_wo_lowhits[k][site][seq_id]





{'RI_3_NODE_4438_length_7624_pol762_H': {'rank': 76, 'pos_w_min': 372}}

Escheria_coli_IAI39_POL_ref [(761, 86)]
RI_1_NODE_15_length_88571_pol762_L_rnr438_I [(578, 90)]
RI_8_NODE_36_length_107729_pol762_L_rnr438_I [(582, 89)]
RI_7_NODE_69_length_56267_pol762_L_rnr438_P [(442, 88)]
RI_8_NODE_76_length_74904_pol762_L_rnr438_P [(559, 87)]
RI_1_NODE_1070_length_18682_pol762_L_rnr438_Na [(409, 83)]
RI_1_NODE_1463_length_15583_pol762_L_rnr438_Na [(442, 87)]
RI_1_NODE_2638_length_10869_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_16567_length_3591_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_38210_length_2110_pol762_L_rnr438_Na [(441, 87)]
RI_1_NODE_81373_length_1293_pol762_L_rnr438_Na [(202, 87)]
RI_2_NODE_11719_length_4423_pol762_L_rnr438_Na [(442, 87)]
RI_2_NODE_27612_length_2569_pol762_L_rnr438_Na [(451, 86)]
RI_3_NODE_1013_length_18274_pol762_L_rnr438_Na [(428, 91)]
RI_5_NODE_17172_length_3280_pol762_L_rnr438_Na [(401, 86)]
RI_5_NODE_22231_length_2797_pol762_L_rnr438_Na [(445, 88)]
RI_6_NODE_488

RI_7_NODE_558_length_24954_pol762_H [(304, 86), (308, 45)]
RI_7_NODE_6741_length_5875_pol762_H [(303, 86), (307, 44), (332, 15)]
RI_8_NODE_125_length_61024_pol762_H [(304, 85), (308, 44), (333, 14)]
RI_8_NODE_150_length_57973_pol762_H [(303, 85), (307, 44), (332, 14)]
RI_8_NODE_342_length_43855_pol762_H [(303, 85), (307, 44)]
RI_8_NODE_394_length_41615_pol762_H [(590, 87), (594, 44), (636, 19)]
RI_8_NODE_451_length_39491_pol762_H [(615, 87), (619, 45), (644, 23)]
RI_8_NODE_3373_length_14800_pol762_H [(396, 91), (400, 46), (425, 27)]
RI_8_NODE_7428_length_9580_pol762_H [(303, 86), (307, 44), (332, 15)]
RI_8_NODE_26768_length_4225_pol762_H [(292, 87), (296, 45), (321, 24)]
RI_2_NODE_51405_length_1730_pol762_F [(286, 87)]
RI_8_NODE_182_length_55288_pol762_F [(293, 59)]
RI_8_NODE_2060_length_19529_pol762_F [(262, 49)]
RI_8_NODE_54482_length_2278_pol762_Y [(382, 91), (386, 46), (413, 26)]
RI_8_NODE_14194_length_6542_pol762_F [(265, 62), (269, 32)]
RI_5_NODE_27622_length_2446_pol762_F [(267,

RI_8_NODE_2060_length_19529_pol762_F [(262, 49)]
RI_8_NODE_54482_length_2278_pol762_Y [(382, 91), (386, 46), (413, 14)]
RI_8_NODE_14194_length_6542_pol762_F [(265, 62), (269, 32)]
RI_5_NODE_27622_length_2446_pol762_F [(267, 28)]

Number of Iterations: 27

{'RI_8_NODE_2304_length_18383_pol762_Y': {'rank': 77, 'pos_w_min': 432}}

Escheria_coli_IAI39_POL_ref [(761, 86)]
RI_1_NODE_15_length_88571_pol762_L_rnr438_I [(578, 90)]
RI_8_NODE_36_length_107729_pol762_L_rnr438_I [(582, 89)]
RI_7_NODE_69_length_56267_pol762_L_rnr438_P [(442, 88)]
RI_8_NODE_76_length_74904_pol762_L_rnr438_P [(559, 87)]
RI_1_NODE_1070_length_18682_pol762_L_rnr438_Na [(409, 83)]
RI_1_NODE_1463_length_15583_pol762_L_rnr438_Na [(442, 87)]
RI_1_NODE_2638_length_10869_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_16567_length_3591_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_38210_length_2110_pol762_L_rnr438_Na [(441, 87)]
RI_1_NODE_81373_length_1293_pol762_L_rnr438_Na [(202, 87)]
RI_2_NODE_11719_length_4423_pol762_L_rnr438_Na [(44

RI_1_NODE_1070_length_18682_pol762_L_rnr438_Na [(409, 83)]
RI_1_NODE_1463_length_15583_pol762_L_rnr438_Na [(442, 87)]
RI_1_NODE_2638_length_10869_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_16567_length_3591_pol762_L_rnr438_Na [(445, 87)]
RI_1_NODE_38210_length_2110_pol762_L_rnr438_Na [(441, 87)]
RI_1_NODE_81373_length_1293_pol762_L_rnr438_Na [(202, 87)]
RI_2_NODE_11719_length_4423_pol762_L_rnr438_Na [(442, 87)]
RI_2_NODE_27612_length_2569_pol762_L_rnr438_Na [(451, 86)]
RI_3_NODE_1013_length_18274_pol762_L_rnr438_Na [(428, 91)]
RI_5_NODE_17172_length_3280_pol762_L_rnr438_Na [(401, 86)]
RI_5_NODE_22231_length_2797_pol762_L_rnr438_Na [(445, 88)]
RI_6_NODE_4881_length_6792_pol762_L_rnr438_Na [(442, 87)]
RI_7_NODE_6171_length_6187_pol762_L_rnr438_Na [(442, 86)]
RI_8_NODE_291_length_46820_pol762_L_rnr438_Na [(444, 87)]
RI_8_NODE_1778_length_21288_pol762_L_rnr438_Na [(424, 91)]
RI_8_NODE_2111_length_19237_pol762_L_rnr438_Na [(444, 87)]
RI_8_NODE_3215_length_15186_pol762_L_rnr438_Na [(438, 87)]


RI_8_NODE_6295_length_10464_pol762_Y [(477, 86), (481, 32)]
RI_8_NODE_7417_length_9589_pol762_Y [(429, 90), (433, 33)]
RI_8_NODE_8959_length_8600_pol762_Y [(449, 92)]
RI_8_NODE_10683_length_7755_pol762_Y [(427, 91), (431, 33)]
RI_8_NODE_31579_length_3711_pol762_Y [(312, 91), (316, 33)]
RI_8_NODE_46574_length_2638_pol762_Y [(396, 92)]
RI_9_NODE_25750_length_2605_pol762_Y [(392, 91), (396, 33)]
RI_9_NODE_88297_length_1186_pol762_Y [(236, 90), (240, 33)]
RI_2_NODE_340_length_33288_pol762_H [(304, 87), (308, 32)]
RI_3_NODE_178_length_37957_pol762_H [(303, 85), (307, 31)]
RI_3_NODE_3136_length_9350_pol762_H [(303, 85), (307, 31)]
RI_3_NODE_4438_length_7624_pol762_H [(304, 87), (308, 32)]
RI_7_NODE_558_length_24954_pol762_H [(304, 86), (308, 32)]
RI_7_NODE_6741_length_5875_pol762_H [(303, 86), (307, 31)]
RI_8_NODE_125_length_61024_pol762_H [(304, 85), (308, 31)]
RI_8_NODE_150_length_57973_pol762_H [(303, 85), (307, 31)]
RI_8_NODE_342_length_43855_pol762_H [(303, 85), (307, 31)]
RI_8_NODE_394

RI_9_NODE_25750_length_2605_pol762_Y [(392, 91)]
RI_9_NODE_88297_length_1186_pol762_Y [(236, 90), (240, 19)]
RI_2_NODE_340_length_33288_pol762_H [(304, 87), (308, 18)]
RI_3_NODE_178_length_37957_pol762_H [(303, 85), (307, 17)]
RI_3_NODE_3136_length_9350_pol762_H [(303, 85), (307, 17)]
RI_3_NODE_4438_length_7624_pol762_H [(304, 87), (308, 18)]
RI_7_NODE_558_length_24954_pol762_H [(304, 86), (308, 18)]
RI_7_NODE_6741_length_5875_pol762_H [(303, 86), (307, 17)]
RI_8_NODE_125_length_61024_pol762_H [(304, 85), (308, 17)]
RI_8_NODE_150_length_57973_pol762_H [(303, 85), (307, 17)]
RI_8_NODE_342_length_43855_pol762_H [(303, 85), (307, 17)]
RI_8_NODE_394_length_41615_pol762_H [(590, 87), (594, 17)]
RI_8_NODE_451_length_39491_pol762_H [(615, 87), (619, 18)]
RI_8_NODE_3373_length_14800_pol762_H [(396, 91)]
RI_8_NODE_7428_length_9580_pol762_H [(303, 86), (307, 17)]
RI_8_NODE_26768_length_4225_pol762_H [(292, 87), (296, 18)]
RI_2_NODE_51405_length_1730_pol762_F [(286, 87)]
RI_8_NODE_182_length_5528

RI_4_NODE_2634_length_10018_pol762_F [(436, 89)]
RI_4_NODE_33046_length_2135_pol762_F [(460, 92)]
RI_5_NODE_28580_length_2395_pol762_F [(337, 92)]
RI_6_NODE_829_length_19189_pol762_F [(529, 92)]
RI_7_NODE_16161_length_3434_pol762_F [(262, 42)]
RI_7_NODE_24616_length_2660_pol762_F [(224, 61)]
RI_8_NODE_41_length_101786_pol762_F [(570, 86)]
RI_8_NODE_2499_length_17595_pol762_F [(484, 92)]
RI_8_NODE_2957_length_15961_pol762_F [(513, 91)]
RI_8_NODE_6314_length_10441_pol762_F [(514, 90)]
RI_8_NODE_6736_length_10102_pol762_F [(413, 90)]
RI_8_NODE_8310_length_8980_pol762_F [(501, 92)]
RI_8_NODE_30712_length_3793_pol762_F [(365, 91)]
RI_9_NODE_687_length_22603_pol762_F [(510, 92)]
RI_9_NODE_20852_length_2966_pol762_F [(298, 77), (302, 2)]
RI_1_NODE_67_length_57010_pol762_Y [(408, 91)]
RI_1_NODE_2353_length_11652_pol762_Y [(428, 91)]
RI_1_NODE_30624_length_2435_pol762_Y [(467, 90)]
RI_2_NODE_5182_length_7325_pol762_Y [(421, 91)]
RI_5_NODE_123_length_43779_pol762_Y [(446, 92)]
RI_5_NODE_64831_le